
<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: n-gram models </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [1]:
import io, sys, math, re
from collections import defaultdict

import numpy as np

In [2]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile
    
    Returns:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [3]:
print("load training set..")
print("\n")
train_data, vocab = load_data("train1.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("valid1.txt")


load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


how : 107
load validation set


In [4]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count 
    
    Returns: 
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab 
    # or have a count that is below mincount
    data_with_unk = []

    ## FILL CODE

    for text in data:

      for  i in range(len(text)):

        if vocab[text[i]]<=mincount:

          text[i]='<unk>'
      data_with_unk.append(text)
    
    return data_with_unk

In [5]:
print("remove rare words")
train_data = remove_rare_words(train_data, vocab, mincount = 1)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
#train_data
print(train_data[0])

remove rare words
['<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>']


In [6]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    n (int): size of the n-gram
    
    Returns:
    proba (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    
    '''
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))
    words_counts=defaultdict(lambda: 0)
    for sentence in data:
        sentence = tuple(sentence)
        ## FILL CODE
        # dict can be indexed by tuples
        # store in the same dict all the ngrams
        # by using the context as a key and the word as a value
        for i in range(len(sentence)):
            for j in range(n):
                if i-j < 0:
                    continue
                counts[sentence[i-j:i]][sentence[i]] +=1

            words_counts[sentence[i]]+=1 
                           

    prob  = defaultdict(lambda: defaultdict(lambda: 0.0))
    # Build the probabilities from the counts
    # Be careful with how you normalize!

    #V=len(words_counts)

    for context in counts.keys():
      
      ## FILL CODE
      word_in_context=0
      for word in counts[context].keys():

        word_in_context+=counts[context][word]

      for word in counts[context].keys():

        prob[context][word]= (counts[context][word])/(word_in_context)   # normalize

    
    return prob

In [7]:
# RUN TO BUILD NGRAM MODEL

n = 2
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)

build ngram model with n =  2


Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [8]:
def get_prob(model, context, w):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context
    
    Retunrs:
    prob(float): probability of this word given the context 
    '''

    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model
    
    ## FILL CODE

    if context in model and w in model[context]:
        return model[context][w]
    else:
        return 0.4*get_prob(model, context[1:], w)

In [9]:
context=('<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>')
w="<unk>"
get_prob(model, context, w)

0.00014227093215330298

In [10]:
import numpy as np

In [33]:
def perplexity(model, data, n):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram
    
    Retunrs:
    prep(float): the preplexity of the model 
    '''
    ## let's create a train vocab to avoid OOV's
    ## FILL CODE
    logperp= 0.0 
    T=0
    for sentence in data:
        sentence = tuple(sentence)
        for i in range(1,len(sentence)):     #('allassan','tony','stark')   
          idx = min(n-1, i)
          context = sentence[i-idx:i]
          logperp += -np.log(get_prob(model,context, sentence[i]))
          T += 1
    perp = np.exp(logperp/T)
    return perp

There are some word which may appear in the valid data but not in the train data. As a result, the probability of that word given a context 

$p(w|context)=0$

as a result the perplexity, can turn to $∞$. We need to perform some smoothing in case this happends

In [34]:
#COMPUTE PERPLEXITY ON VALIDATION SET
print("The perplexity is", perplexity(model, valid_data, n=2))

The perplexity is 53.582886955857006


In [35]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context 
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and 
    thier probabilites
    
    Retunrs:
    words_and_probs(dic): {word: probability of word given context}
    
    '''
    # code a recursive function over context
    # to find the longest available ngram
    
    ## FILL CODE
    
    if context in model:
        return model[context]
    else:
        return get_proba_distrib(model, context[1:])
    return 0.0

In [39]:
def generate(model):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model. 
    

    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    sentence = ["<s>"]
    while sentence[-1] != "</s>" and len(sentence)<100:
        ## FILL CODE
        prob = get_proba_distrib(model, tuple(sentence))
        w = np.random.choice((list(prob.keys())), 1, p = list(prob.values()))
        sentence.append(w[0])
        
    return sentence

In [50]:
# GENERATE A SENTENCE FROM THE MODEL

print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', 'tom', 'and', 'mary', 'they', "would've", 'done', 'that.', '</s>']


Some  thought: Trying to normalize the model probability using laplacian smoothing with $α=1$, gave higher perplexity

$proba=\frac{c\left(w_{t}w_{t-1}\right)+1}{c\left(w_{t-1}\right)+V}$


$V$ been the size of the vocabulary

Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.